## Libs

In [1]:
import matplotlib as matpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

# Using the Tensorflow backend (default).
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from tensorflow import set_random_seed
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping

# advanced plotting
import seaborn as sns
plt.style.use('seaborn-darkgrid')
%matplotlib inline

Using TensorFlow backend.


## Data

In [2]:
%store -r data_EMB

In [3]:
mm = MinMaxScaler(feature_range = (0,1))
dataset = mm.fit_transform(data_EMB)

In [4]:
split = 0.6

train_size = int(len(dataset) * split)
#validation
test_size = len(dataset) - train_size

train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]

print("training, test set: " + str((len(train), len(test))))

training, test set: (313, 209)


In [5]:
def input_dataset(dataset, window):
    data_X, data_y = [], []
    for i in range(len(dataset) - window - 1):
        a = dataset[i:(i + window), 0]
        data_X.append(a)
        data_y.append(dataset[i + window, 0])
    return(np.array(data_X), np.array(data_y))


In [6]:
# New testing and training sets for rolling forecast.
window = 1
train_X, train_Y = input_dataset(train, window)
test_X, test_Y = input_dataset(test, window)
print("Original train shape:")
print(train_X.shape)

# Reshape input data to match Keras format.
train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))
print("New train shape:")
print(train_X.shape)

Original train shape:
(311, 1)
New train shape:
(311, 1, 1)


## 1 Epoch

In [7]:
def fit_LSTM(train_X, train_Y, window = 1, neurons=128):
    set_random_seed(3)
    model = Sequential()
    
    model.add(LSTM(neurons, 
                   input_shape = (1, window)
                  ))
    model.add(Dense(1))
    model.compile(loss = 'mean_squared_error', 
                  optimizer = 'adam')
    earlyStop=EarlyStopping(monitor='val_loss',verbose=2,patience=15)
    model.fit(train_X, 
              train_Y, 
              epochs = 1, 
              batch_size = 1,
              shuffle = False
             # verbose = 2
             )
    
    return(model)

# Fit the first model.
model1 = fit_LSTM(train_X, train_Y, window)

W0919 05:09:37.166362 4661663168 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0919 05:09:37.168513 4661663168 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0919 05:09:37.172357 4661663168 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0919 05:09:37.434950 4661663168 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0919 05:09:37.778733 4661663168 deprecation.py:323] From //anaconda3/li

Epoch 1/1
311/311 [==============================] - 5s 16ms/step - loss: 0.0011


In [9]:
def prediction_score(model, X, Y):
    # Make predictions using input data
    pred = mm.inverse_transform(model.predict(X))
    # Show Y on original scale
    original_data = mm.inverse_transform([Y])
    # RMSE.
    score = math.sqrt(mean_squared_error(original_data[0], pred[:, 0]))
    return(score, pred)

In [9]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)
print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 4.76 RMSE


## 1000 Epochs

In [15]:
def fit_LSTM(train_X, train_Y, window = 1, neurons=128):
    set_random_seed(3)
    model = Sequential()
    
    model.add(LSTM(neurons, 
                   input_shape = (1, window)
                  ))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "adam")
    earlyStop=EarlyStopping(monitor="val_loss",verbose=2,patience=15)
    model.fit(train_X, 
              train_Y, 
              epochs = 1000, 
              batch_size = 1,
              shuffle = False
             # verbose = 2
             )
    
    return(model)

# Fit the first model.
model1 = fit_LSTM(train_X, train_Y, window)

Epoch 1/1000
311/311 [==============================] - 5s 16ms/step - loss: 0.0012
Epoch 2/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0045
Epoch 3/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0021
Epoch 4/1000
311/311 [==============================] - 3s 8ms/step - loss: 0.0011
Epoch 5/1000
311/311 [==============================] - 3s 8ms/step - loss: 6.6095e-04
Epoch 6/1000
311/311 [==============================] - 3s 8ms/step - loss: 5.1227e-04
Epoch 7/1000
311/311 [==============================] - 3s 8ms/step - loss: 4.6468e-04
Epoch 8/1000
311/311 [==============================] - 3s 8ms/step - loss: 4.5263e-04
Epoch 9/1000
311/311 [==============================] - 3s 8ms/step - loss: 4.5291e-04
Epoch 10/1000
311/311 [==============================] - 3s 8ms/step - loss: 4.5719e-04
Epoch 11/1000
311/311 [==============================] - 3s 8ms/step - loss: 4.6215e-04
Epoch 12/1000
311/311 [==============================] -

311/311 [==============================] - 3s 8ms/step - loss: 4.0090e-04
Epoch 187/1000
311/311 [==============================] - 3s 8ms/step - loss: 4.0070e-04
Epoch 188/1000
311/311 [==============================] - 3s 8ms/step - loss: 4.0057e-04
Epoch 189/1000
311/311 [==============================] - 3s 9ms/step - loss: 4.0044e-04
Epoch 190/1000
311/311 [==============================] - 3s 9ms/step - loss: 4.0024e-04
Epoch 191/1000
311/311 [==============================] - 3s 8ms/step - loss: 3.9998e-04
Epoch 192/1000
311/311 [==============================] - 3s 9ms/step - loss: 3.9986e-04
Epoch 193/1000
311/311 [==============================] - 3s 8ms/step - loss: 3.9971e-04
Epoch 194/1000
311/311 [==============================] - 3s 8ms/step - loss: 3.9953e-04
Epoch 195/1000
311/311 [==============================] - 3s 8ms/step - loss: 3.9932e-04
Epoch 196/1000
311/311 [==============================] - 3s 8ms/step - loss: 3.9925e-04
Epoch 197/1000
311/311 [============

311/311 [==============================] - 3s 9ms/step - loss: 3.7545e-04
Epoch 370/1000
311/311 [==============================] - 3s 9ms/step - loss: 3.7544e-04
Epoch 371/1000
311/311 [==============================] - 3s 9ms/step - loss: 3.7550e-04
Epoch 372/1000
311/311 [==============================] - 3s 9ms/step - loss: 3.7547e-04
Epoch 373/1000
311/311 [==============================] - 3s 8ms/step - loss: 3.7554e-04
Epoch 374/1000
311/311 [==============================] - 3s 8ms/step - loss: 3.7556e-04
Epoch 375/1000
311/311 [==============================] - 3s 8ms/step - loss: 3.7562e-04
Epoch 376/1000
311/311 [==============================] - 3s 8ms/step - loss: 3.7557e-04
Epoch 377/1000
311/311 [==============================] - 3s 8ms/step - loss: 3.7564e-04
Epoch 378/1000
311/311 [==============================] - 3s 8ms/step - loss: 3.7559e-04
Epoch 379/1000
311/311 [==============================] - 3s 9ms/step - loss: 3.7571e-04
Epoch 380/1000
311/311 [============

311/311 [==============================] - 3s 8ms/step - loss: 3.7487e-04
Epoch 462/1000
311/311 [==============================] - 3s 8ms/step - loss: 3.7495e-04
Epoch 463/1000
311/311 [==============================] - 3s 8ms/step - loss: 3.7474e-04
Epoch 464/1000
311/311 [==============================] - 3s 9ms/step - loss: 3.7483e-04
Epoch 465/1000
311/311 [==============================] - 3s 9ms/step - loss: 3.7479e-04
Epoch 466/1000
311/311 [==============================] - 3s 9ms/step - loss: 3.7457e-04
Epoch 467/1000
311/311 [==============================] - 3s 10ms/step - loss: 3.7467e-04
Epoch 468/1000
311/311 [==============================] - 3s 10ms/step - loss: 3.7464e-04
Epoch 469/1000
311/311 [==============================] - 3s 11ms/step - loss: 3.7457e-04
Epoch 470/1000
311/311 [==============================] - 3s 10ms/step - loss: 3.7439e-04
Epoch 471/1000
311/311 [==============================] - 3s 10ms/step - loss: 3.7445e-04
Epoch 472/1000
311/311 [=======

311/311 [==============================] - 3s 8ms/step - loss: 3.6963e-04
Epoch 554/1000
311/311 [==============================] - 3s 8ms/step - loss: 3.6967e-04
Epoch 555/1000
311/311 [==============================] - 3s 8ms/step - loss: 3.6949e-04
Epoch 556/1000
311/311 [==============================] - 2s 8ms/step - loss: 3.6949e-04
Epoch 557/1000
311/311 [==============================] - 2s 8ms/step - loss: 3.6956e-04
Epoch 558/1000
311/311 [==============================] - 2s 8ms/step - loss: 3.6943e-04
Epoch 559/1000
311/311 [==============================] - 2s 8ms/step - loss: 3.6932e-04
Epoch 560/1000
311/311 [==============================] - 2s 8ms/step - loss: 3.6934e-04
Epoch 561/1000
311/311 [==============================] - 2s 8ms/step - loss: 3.6939e-04
Epoch 562/1000
311/311 [==============================] - 2s 8ms/step - loss: 3.6940e-04
Epoch 563/1000
311/311 [==============================] - 2s 8ms/step - loss: 3.6923e-04
Epoch 564/1000
311/311 [============

311/311 [==============================] - 3s 9ms/step - loss: 3.6457e-04
Epoch 736/1000
311/311 [==============================] - 3s 9ms/step - loss: 3.6446e-04
Epoch 737/1000
311/311 [==============================] - 3s 10ms/step - loss: 3.6416e-04
Epoch 738/1000
311/311 [==============================] - 3s 9ms/step - loss: 3.6413e-04
Epoch 739/1000
311/311 [==============================] - 3s 9ms/step - loss: 3.6442e-04
Epoch 740/1000
311/311 [==============================] - 3s 9ms/step - loss: 3.6443e-04
Epoch 741/1000
311/311 [==============================] - 3s 9ms/step - loss: 3.6423e-04
Epoch 742/1000
311/311 [==============================] - 3s 9ms/step - loss: 3.6423e-04
Epoch 743/1000
311/311 [==============================] - 3s 9ms/step - loss: 3.6431e-04
Epoch 744/1000
311/311 [==============================] - 3s 9ms/step - loss: 3.6387e-04
Epoch 745/1000
311/311 [==============================] - 3s 9ms/step - loss: 3.6406e-04
Epoch 746/1000
311/311 [===========

311/311 [==============================] - 2s 8ms/step - loss: 3.6101e-04
Epoch 919/1000
311/311 [==============================] - 2s 8ms/step - loss: 3.6084e-04
Epoch 920/1000
311/311 [==============================] - 2s 8ms/step - loss: 3.6064e-04
Epoch 921/1000
311/311 [==============================] - 2s 8ms/step - loss: 3.6090e-04
Epoch 922/1000
311/311 [==============================] - 2s 8ms/step - loss: 3.6098e-04
Epoch 923/1000
311/311 [==============================] - 2s 8ms/step - loss: 3.6082e-04
Epoch 924/1000
311/311 [==============================] - 2s 8ms/step - loss: 3.6079e-04
Epoch 925/1000
311/311 [==============================] - 2s 8ms/step - loss: 3.6084e-04
Epoch 926/1000
311/311 [==============================] - 2s 8ms/step - loss: 3.6092e-04
Epoch 927/1000
311/311 [==============================] - 2s 8ms/step - loss: 3.6078e-04
Epoch 928/1000
311/311 [==============================] - 2s 8ms/step - loss: 3.6067e-04
Epoch 929/1000
311/311 [============

In [16]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)

print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 3.57 RMSE


## 2000 Epochs

In [17]:
def fit_LSTM(train_X, train_Y, window = 1, neurons=128):
    set_random_seed(3)
    model = Sequential()
    
    model.add(LSTM(neurons, 
                   input_shape = (1, window)
                  ))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "adam")
    earlyStop=EarlyStopping(monitor="val_loss",verbose=2,patience=15)
    model.fit(train_X, 
              train_Y, 
              epochs = 2000, 
              batch_size = 1,
              shuffle = False
             # verbose = 2
             )
    
    return(model)

# Fit the first model.
model1 = fit_LSTM(train_X, train_Y, window)

Epoch 1/2000
311/311 [==============================] - 6s 20ms/step - loss: 0.0011
Epoch 2/2000
311/311 [==============================] - 2s 8ms/step - loss: 0.0040
Epoch 3/2000
311/311 [==============================] - 3s 8ms/step - loss: 0.0020
Epoch 4/2000
311/311 [==============================] - 2s 8ms/step - loss: 0.0010
Epoch 5/2000
311/311 [==============================] - 2s 8ms/step - loss: 6.4891e-04
Epoch 6/2000
311/311 [==============================] - 3s 8ms/step - loss: 5.1086e-04
Epoch 7/2000
311/311 [==============================] - 3s 8ms/step - loss: 4.6639e-04
Epoch 8/2000
311/311 [==============================] - 3s 9ms/step - loss: 4.5526e-04
Epoch 9/2000
311/311 [==============================] - 3s 9ms/step - loss: 4.5567e-04
Epoch 10/2000
311/311 [==============================] - 3s 9ms/step - loss: 4.5972e-04
Epoch 11/2000
311/311 [==============================] - 3s 9ms/step - loss: 4.6424e-04
Epoch 12/2000
311/311 [==============================] -

311/311 [==============================] - 3s 8ms/step - loss: 4.0586e-04
Epoch 186/2000
311/311 [==============================] - 3s 8ms/step - loss: 4.0569e-04
Epoch 187/2000
311/311 [==============================] - 3s 9ms/step - loss: 4.0544e-04
Epoch 188/2000
311/311 [==============================] - 3s 10ms/step - loss: 4.0531e-04
Epoch 189/2000
311/311 [==============================] - 3s 11ms/step - loss: 4.0511e-04
Epoch 190/2000
311/311 [==============================] - 3s 11ms/step - loss: 4.0491e-04
Epoch 191/2000
311/311 [==============================] - 3s 10ms/step - loss: 4.0471e-04
Epoch 192/2000
311/311 [==============================] - 4s 11ms/step - loss: 4.0460e-04
Epoch 193/2000
311/311 [==============================] - 3s 11ms/step - loss: 4.0436e-04
Epoch 194/2000
311/311 [==============================] - 4s 13ms/step - loss: 4.0423e-04
Epoch 195/2000
311/311 [==============================] - 4s 14ms/step - loss: 4.0397e-04
Epoch 196/2000
311/311 [====

Epoch 277/2000
311/311 [==============================] - 2s 8ms/step - loss: 3.8851e-04
Epoch 278/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.8837e-04
Epoch 279/2000
311/311 [==============================] - 2s 8ms/step - loss: 3.8819e-04
Epoch 280/2000
311/311 [==============================] - 2s 8ms/step - loss: 3.8807e-04
Epoch 281/2000
311/311 [==============================] - 2s 8ms/step - loss: 3.8787e-04
Epoch 282/2000
311/311 [==============================] - 2s 8ms/step - loss: 3.8769e-04
Epoch 283/2000
311/311 [==============================] - 2s 8ms/step - loss: 3.8753e-04
Epoch 284/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.8733e-04
Epoch 285/2000
311/311 [==============================] - 2s 8ms/step - loss: 3.8714e-04
Epoch 286/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.8696e-04
Epoch 287/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.8676e-04
Epoch 288/2000
311/31

311/311 [==============================] - 2s 8ms/step - loss: 3.7325e-04
Epoch 370/2000
311/311 [==============================] - 2s 8ms/step - loss: 3.7326e-04
Epoch 371/2000
311/311 [==============================] - 2s 8ms/step - loss: 3.7329e-04
Epoch 372/2000
311/311 [==============================] - 2s 8ms/step - loss: 3.7329e-04
Epoch 373/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.7333e-04
Epoch 374/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.7334e-04
Epoch 375/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.7338e-04
Epoch 376/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.7332e-04
Epoch 377/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.7342e-04
Epoch 378/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.7334e-04
Epoch 379/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.7344e-04
Epoch 380/2000
311/311 [============

311/311 [==============================] - 2s 8ms/step - loss: 3.6819e-04
Epoch 554/2000
311/311 [==============================] - 2s 8ms/step - loss: 3.6823e-04
Epoch 555/2000
311/311 [==============================] - 2s 8ms/step - loss: 3.6818e-04
Epoch 556/2000
311/311 [==============================] - 2s 8ms/step - loss: 3.6802e-04
Epoch 557/2000
311/311 [==============================] - 2s 8ms/step - loss: 3.6818e-04
Epoch 558/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.6800e-04
Epoch 559/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.6804e-04
Epoch 560/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.6797e-04
Epoch 561/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.6801e-04
Epoch 562/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.6788e-04
Epoch 563/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.6789e-04
Epoch 564/2000
311/311 [============

...

311/311 [==============================] - 3s 8ms/step - loss: 3.5872e-04
Epoch 1377/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.5862e-04
Epoch 1378/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.5883e-04
Epoch 1379/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.5874e-04
Epoch 1380/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5872e-04
Epoch 1381/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5860e-04
Epoch 1382/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5870e-04
Epoch 1383/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5864e-04
Epoch 1384/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5880e-04
Epoch 1385/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5872e-04
Epoch 1386/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5863e-04
Epoch 1387/2000
311/311 [=

Epoch 1467/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.5801e-04
Epoch 1468/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.5817e-04
Epoch 1469/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.5808e-04
Epoch 1470/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5801e-04
Epoch 1471/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5778e-04
Epoch 1472/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5790e-04
Epoch 1473/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5817e-04
Epoch 1474/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5803e-04
Epoch 1475/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5793e-04
Epoch 1476/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5783e-04
Epoch 1477/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5792e-04
Epoch 1478

311/311 [==============================] - 3s 8ms/step - loss: 3.5674e-04
Epoch 1648/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.5687e-04A: 0s - loss: 3.6
Epoch 1649/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.5675e-04
Epoch 1650/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.5661e-04
Epoch 1651/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5667e-04
Epoch 1652/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5685e-04
Epoch 1653/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5672e-04
Epoch 1654/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5670e-04
Epoch 1655/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5690e-04
Epoch 1656/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5666e-04
Epoch 1657/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5677e-04
Epoch 165

311/311 [==============================] - 3s 8ms/step - loss: 3.5612e-04
Epoch 1739/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.5607e-04
Epoch 1740/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.5607e-04
Epoch 1741/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.5625e-04
Epoch 1742/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5607e-04
Epoch 1743/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5593e-04
Epoch 1744/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5597e-04
Epoch 1745/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5594e-04
Epoch 1746/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5599e-04
Epoch 1747/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.5597e-04
Epoch 1748/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5614e-04
Epoch 1749/2000
311/311 [=

311/311 [==============================] - 3s 8ms/step - loss: 3.5458e-04
Epoch 1921/2000
311/311 [==============================] - 3s 8ms/step - loss: 3.5450e-04
Epoch 1922/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5454e-04
Epoch 1923/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5469e-04
Epoch 1924/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5461e-04
Epoch 1925/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5477e-04
Epoch 1926/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5468e-04
Epoch 1927/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5456e-04
Epoch 1928/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5463e-04
Epoch 1929/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5434e-04
Epoch 1930/2000
311/311 [==============================] - 3s 9ms/step - loss: 3.5453e-04
Epoch 1931/2000
311/311 [=

In [18]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)
print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 4.02 RMSE


## 2500 Epochs

In [7]:
def fit_LSTM(train_X, train_Y, window = 1, neurons=128):
    set_random_seed(3)
    model = Sequential()
    
    model.add(LSTM(neurons, 
                   input_shape = (1, window)
                  ))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "adam")
    earlyStop=EarlyStopping(monitor="val_loss",verbose=2,patience=15)
    model.fit(train_X, 
              train_Y, 
              epochs = 2500, 
              batch_size = 1,
              shuffle = False
             # verbose = 2
             )
    
    return(model)

# Fit the first model.
model1 = fit_LSTM(train_X, train_Y, window)

W0919 12:11:41.348356 4619789760 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0919 12:11:41.350569 4619789760 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0919 12:11:41.353188 4619789760 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0919 12:11:41.530133 4619789760 deprecation_wrapper.py:119] From //anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0919 12:11:41.740127 4619789760 deprecation.py:323] From //anaconda3/li

Epoch 1/2500
311/311 [==============================] - 3s 11ms/step - loss: 0.0012
Epoch 2/2500
311/311 [==============================] - 2s 5ms/step - loss: 0.0042
Epoch 3/2500
311/311 [==============================] - 2s 6ms/step - loss: 0.0020
Epoch 4/2500
311/311 [==============================] - 2s 5ms/step - loss: 0.0011
Epoch 5/2500
311/311 [==============================] - 2s 5ms/step - loss: 6.6267e-04
Epoch 6/2500
311/311 [==============================] - 2s 5ms/step - loss: 5.1544e-04
Epoch 7/2500
311/311 [==============================] - 2s 5ms/step - loss: 4.6719e-04
Epoch 8/2500
311/311 [==============================] - 2s 5ms/step - loss: 4.5452e-04
Epoch 9/2500
311/311 [==============================] - 2s 6ms/step - loss: 4.5429e-04
Epoch 10/2500
311/311 [==============================] - 2s 6ms/step - loss: 4.5807e-04
Epoch 11/2500
311/311 [==============================] - 2s 7ms/step - loss: 4.6253e-04
Epoch 12/2500
311/311 [==============================] -

311/311 [==============================] - 2s 6ms/step - loss: 3.9425e-04
Epoch 95/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.9471e-04
Epoch 96/2500
311/311 [==============================] - 2s 7ms/step - loss: 3.9528e-04
Epoch 97/2500
311/311 [==============================] - 3s 9ms/step - loss: 3.9557e-04
Epoch 98/2500
311/311 [==============================] - 2s 7ms/step - loss: 3.9616e-04
Epoch 99/2500
311/311 [==============================] - 2s 7ms/step - loss: 3.9654e-04
Epoch 100/2500
311/311 [==============================] - 2s 7ms/step - loss: 3.9709e-04
Epoch 101/2500
311/311 [==============================] - 2s 7ms/step - loss: 3.9739e-04
Epoch 102/2500
311/311 [==============================] - 2s 7ms/step - loss: 3.9791e-04
Epoch 103/2500
311/311 [==============================] - 2s 7ms/step - loss: 3.9826e-04
Epoch 104/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.9872e-04
Epoch 105/2500
311/311 [=================

311/311 [==============================] - 2s 6ms/step - loss: 4.0172e-04
Epoch 187/2500
311/311 [==============================] - 2s 6ms/step - loss: 4.0166e-04
Epoch 188/2500
311/311 [==============================] - 2s 6ms/step - loss: 4.0143e-04
Epoch 189/2500
311/311 [==============================] - 2s 6ms/step - loss: 4.0131e-04
Epoch 190/2500
311/311 [==============================] - 2s 6ms/step - loss: 4.0116e-04
Epoch 191/2500
311/311 [==============================] - 2s 6ms/step - loss: 4.0102e-04
Epoch 192/2500
311/311 [==============================] - 2s 6ms/step - loss: 4.0096e-04
Epoch 193/2500
311/311 [==============================] - 2s 6ms/step - loss: 4.0084e-04
Epoch 194/2500
311/311 [==============================] - 2s 6ms/step - loss: 4.0064e-04
Epoch 195/2500
311/311 [==============================] - 2s 6ms/step - loss: 4.0048e-04
Epoch 196/2500
311/311 [==============================] - 2s 6ms/step - loss: 4.0031e-04
Epoch 197/2500
311/311 [============

311/311 [==============================] - 2s 6ms/step - loss: 3.8422e-04
Epoch 279/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.8355e-04
Epoch 280/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.8286e-04
Epoch 281/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.8210e-04
Epoch 282/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.8135e-04
Epoch 283/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.8052e-04
Epoch 284/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.7979e-04
Epoch 285/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.7892e-04
Epoch 286/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.7821e-04
Epoch 287/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.7736e-04
Epoch 288/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.7675e-04
Epoch 289/2500
311/311 [============

...

311/311 [==============================] - 2s 7ms/step - loss: 3.5496e-04
Epoch 2106/2500
311/311 [==============================] - 2s 7ms/step - loss: 3.5493e-04
Epoch 2107/2500
311/311 [==============================] - 2s 7ms/step - loss: 3.5492e-04
Epoch 2108/2500
311/311 [==============================] - 2s 7ms/step - loss: 3.5508e-04
Epoch 2109/2500
311/311 [==============================] - 2s 7ms/step - loss: 3.5487e-04
Epoch 2110/2500
311/311 [==============================] - 2s 7ms/step - loss: 3.5440e-04
Epoch 2111/2500
311/311 [==============================] - 2s 7ms/step - loss: 3.5476e-04
Epoch 2112/2500
311/311 [==============================] - 2s 7ms/step - loss: 3.5468e-04
Epoch 2113/2500
311/311 [==============================] - 2s 7ms/step - loss: 3.5474e-04
Epoch 2114/2500
311/311 [==============================] - 2s 7ms/step - loss: 3.5476e-04
Epoch 2115/2500
311/311 [==============================] - 2s 7ms/step - loss: 3.5468e-04
Epoch 2116/2500
311/311 [=

311/311 [==============================] - 2s 6ms/step - loss: 3.5442e-04
Epoch 2197/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5429e-04
Epoch 2198/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5428e-04
Epoch 2199/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5433e-04
Epoch 2200/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5424e-04
Epoch 2201/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5428e-04
Epoch 2202/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5424e-04
Epoch 2203/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5415e-04
Epoch 2204/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5398e-04
Epoch 2205/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5431e-04
Epoch 2206/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5427e-04
Epoch 2207/2500
311/311 [=

311/311 [==============================] - 2s 6ms/step - loss: 3.5356e-04
Epoch 2288/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5389e-04
Epoch 2289/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5367e-04
Epoch 2290/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5388e-04
Epoch 2291/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5383e-04
Epoch 2292/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5375e-04
Epoch 2293/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5388e-04
Epoch 2294/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5369e-04
Epoch 2295/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5366e-04
Epoch 2296/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5377e-04
Epoch 2297/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5360e-04
Epoch 2298/2500
311/311 [=

311/311 [==============================] - 2s 6ms/step - loss: 3.5289e-04
Epoch 2379/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5295e-04
Epoch 2380/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5326e-04
Epoch 2381/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5312e-04
Epoch 2382/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5325e-04
Epoch 2383/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5311e-04
Epoch 2384/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5294e-04
Epoch 2385/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5310e-04
Epoch 2386/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5301e-04
Epoch 2387/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5319e-04
Epoch 2388/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5301e-04
Epoch 2389/2500
311/311 [=

311/311 [==============================] - 2s 6ms/step - loss: 3.5241e-04
Epoch 2470/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5258e-04
Epoch 2471/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5246e-04
Epoch 2472/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5259e-04
Epoch 2473/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5245e-04
Epoch 2474/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5236e-04
Epoch 2475/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5236e-04
Epoch 2476/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5233e-04
Epoch 2477/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5261e-04
Epoch 2478/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5238e-04
Epoch 2479/2500
311/311 [==============================] - 2s 6ms/step - loss: 3.5260e-04
Epoch 2480/2500
311/311 [=

In [10]:
rmse_test, test_pred = prediction_score(model1, test_X, test_Y)
print("Testing score: %.2f RMSE" % rmse_test)

Testing score: 4.24 RMSE
